In [9]:
import train as train
import generate
import eval
from modules.config import read_config
import torch
import numpy as np

In [10]:
BASE_DIR = "./experiments/model-test"
PROFILE = "no-layer-sum"

In [11]:
configs = read_config(f"{BASE_DIR}/config.ini")
dataset_configs = read_config(f"{BASE_DIR}/dataset_config.ini")

In [12]:
config = configs[PROFILE]
NAME = config["NAME"]
dataset_config = dataset_configs[config["DATASET"]]

In [13]:
GENERATE = True
TRAIN = False
EVAL = True

In [14]:
if GENERATE:
    generate.gen(dataset_config, f"{BASE_DIR}/datasets")

In [15]:
if TRAIN:
    train.train(config, f"{BASE_DIR}")

In [16]:
if EVAL:
    eval.eval(config, dataset_config, f"{BASE_DIR}")

OSError: ./experiments/model-test/datasets/sum-50k/val.csv not found.